from Qian

In [0]:
%pip install unstructured-client -q
%pip install databricks-sdk -U -q
%pip install tiktoken -U -q
%pip install azure-ai-documentintelligence==1.0.0b3 -q
dbutils.library.restartPython()

# Unstructured Serverless API

In [0]:
from unstructured_client import UnstructuredClient
from unstructured_client.models import operations, shared

client = UnstructuredClient(
    api_key_auth=dbutils.secrets.get(scope="tokens_fins_ssa_qyu",                   
                                     key='unstructed_serverless_token'),
    server_url="https://api.unstructuredapp.io",
)


## Parsing PDF elements

In [0]:
filename = "/Volumes/qyu_test/genai_dev/pdfs/2022-09-07_42214-TheGrove_CBREOM_2.pdf"
with open(filename, "rb") as f:
    data = f.read()

req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=shared.Files(
            content=data,
            file_name=filename,
        ),
        strategy=shared.Strategy.AUTO,  
        languages=['eng'],
    ),
)

try:
    res = client.general.partition(request=req)
    print(res.elements[0])
except Exception as e:
    print(e)

In [0]:
import pandas as pd

df_elements = pd.DataFrame([e for e in res.elements])
display(df_elements)

In [0]:
df_elements['type'].value_counts()

In [0]:
df_tables = df_elements[df_elements['type'] == 'Table'].reset_index(drop=True)
df_tables['html_table'] = df_tables['metadata'].apply(lambda x: x['text_as_html'])
df_tables['filename'] = df_tables['metadata'].apply(lambda x: x['filename'])
display(df_tables)


## Add Table's Parent Element

In [0]:
df_elements['metadata'][1]

In [0]:
df_elements['page_number'] = df_elements['metadata'].apply(lambda x: x['page_number'])
df_elements['parent_id'] = df_elements['metadata'].apply(lambda x: x.get('parent_id', None))
df_elements_parents = df_elements.merge(df_elements[['element_id', 'type', 'text', 'page_number']],
                                    left_on='parent_id',
                                    right_on='element_id',
                                    how='left',
                                    suffixes=('', '_parent'))
display(df_elements_parents)

In [0]:
df_tables_parents = df_elements_parents[df_elements_parents['type'] == 'Table'].reset_index(drop=True)
df_tables_parents['html_table'] = df_tables_parents['metadata'].apply(lambda x: x['text_as_html'])
df_tables_parents['filename'] = df_tables['metadata'].apply(lambda x: x['filename'])
display(df_tables_parents)

In [0]:
%sql
USE CATALOG qyu_test;
USE SCHEMA genai_dev;


## Save parsed the data to a delta table

In [0]:
# convert pandas dataframe df_tables_parents to spark dataframe and save to a delta table
df_tables_parents_spark = spark.createDataFrame(df_tables_parents)
df_tables_parents_spark.write \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .saveAsTable('unstructured_serverless_extracted_tables')


## Display Table Title, Page Number and the Table

In [0]:
for idx, row in df_tables_parents.iterrows():
    print(f"============= Table INFO: {row['type_parent']} - {row['text_parent']} at Page: {row['page_number']} ==============")
    print(displayHTML(row['html_table']))
    print('\n\n')


## Save the html Tables to Panda DataFrames, spark dataframe, then to delta tables

In [0]:
df_tables_parents_spark = spark.table("unstructured_serverless_extracted_tables")
display(df_tables_parents_spark)

In [0]:
import re
import pandas as pd

tables_from_pdf = {}
for idx, row in df_tables_parents_spark.toPandas().iterrows():
    if row['text_parent']: 
        name = re.sub(r'\s+', '_', row['text_parent']) 
    else:
        name = 'untitled'

    name = f"{name}_pg{row['page_number']}"
    print(f"table_name: {name}")
    df = pd.read_html(row['html_table'])[0]
    tables_from_pdf[name] = df

In [0]:
tables_from_pdf['EXPENSES_pg46']

In [0]:
tables_from_pdf['Rent_Roll_as_of_October_1,_2022_pg47']

In [0]:
tables_from_pdf['Cash_Flow_Projections_pg53']


## Select important tables to save to delta table

* Cleaning will probably need to performed, for example, in below example, there # in the column name

In [0]:
display(tables_from_pdf['Competitive_Area_Retail_pg39'])

In [0]:
def create_valid_column_names(columns):
    valid_columns = []
    for col_name in columns:
        # Replace invalid characters with underscores and strip spaces
        valid_name = re.sub('[^a-zA-Z0-9_]', '_', col_name).strip('_')
        valid_columns.append(valid_name)
    return valid_columns

In [0]:
df = spark.createDataFrame(tables_from_pdf['Competitive_Area_Retail_pg39'])
new_cols = create_valid_column_names(df.columns)
new_cols

In [0]:
df_renamed = df.toDF(*new_cols)
display(df_renamed)

In [0]:
df_renamed.write \
    .mode('overwrite') \
    .option('overwriteSchema', 'true') \
    .saveAsTable('Competitive_Area_Retail_pg39')